In [1]:
import sys
import os

# Move up one level so Python can find config.py
sys.path.append(os.path.abspath(".."))


import psycopg2
from config import get_db_connection
import pandas as pd


In [2]:
# Load CSV from /data
df = pd.read_csv("../data/train.csv", encoding="ISO-8859-1")

In [3]:
# Clean column names
df.columns = [col.strip().lower().replace(" ", "_").replace("-", "_") for col in df.columns]


In [4]:
# Preview
df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [5]:
from config import get_db_connection

# Test the connection
conn = get_db_connection()
if conn:
    print("✅ Database connection successful!")
    conn.close()
else:
    print("❌ Database connection failed. Check your credentials in config.py.")


✅ Database connection successful!


In [6]:
conn = get_db_connection()
cursor = conn.cursor()

In [7]:

try: 
    # Drop if exists (for reruns)
    cursor.execute("DROP TABLE IF EXISTS stg_superstore;")
except psycopg2.Error as e:
    print(e)

In [8]:

try: 
    # Create stg_superstore table
    create_table_query = """
    CREATE TABLE stg_superstore (
    row_id INT,
    order_id VARCHAR,
    order_date DATE,
    ship_date DATE,
    ship_mode VARCHAR,
    customer_id VARCHAR,
    customer_name VARCHAR,
    segment VARCHAR,
    country VARCHAR,
    city VARCHAR,
    state VARCHAR,
    postal_code VARCHAR,
    region VARCHAR,
    product_id VARCHAR,
    category VARCHAR,
    sub_category VARCHAR,
    product_name VARCHAR,
    sales NUMERIC
    );
    """
    cursor.execute(create_table_query)
    print("✅ Table 'stg_superstore' created successfully!")
except psycopg2.Error as e:
    print(e)

✅ Table 'stg_superstore' created successfully!


In [9]:
conn.commit()
cursor.close()
conn.close()

print("✅ Staging table 'stg_superstore' created successfully.")

✅ Staging table 'stg_superstore' created successfully.


In [10]:
# Reconnect
conn = get_db_connection()
cursor = conn.cursor()

In [16]:
# Convert all numpy types to native Python types
df = df.astype({
    'row_id': int,
    'order_id': str,
    'order_date': str,
    'ship_date': str,
    'ship_mode': str,
    'customer_id': str,
    'customer_name': str,
    'segment': str,
    'country': str,
    'city': str,
    'state': str,
    'postal_code': str,
    'region': str,
    'product_id': str,
    'category': str,
    'sub_category': str,
    'product_name': str,
    'sales': float
})


In [17]:
# Convert to list of tuples
rows = [tuple(row) for row in df.to_numpy()]

In [ ]:
rows

In [21]:

try: 
    # Insert data for stg_superstore table
    insert_table_query = """
    INSERT INTO stg_superstore (
        row_id, order_id, order_date, ship_date, ship_mode, customer_id,
        customer_name, segment, country, city, state, postal_code, region,
        product_id, category, sub_category, product_name, sales
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    );
    """
    cursor.executemany(insert_table_query, rows)
    print("✅ Inserted data into the 'stg_superstore' table successfully!")
except psycopg2.Error as e:
    print(e)

✅ Inserted data into the 'stg_superstore' table successfully!


In [22]:
# Commit and close
conn.commit()
cursor.close()
conn.close()

print("✅ Inserted data into the 'stg_superstore' successfully in PostgreSQL!")

✅ Inserted data into the 'stg_superstore' successfully in PostgreSQL!
